In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import noise
from numba import njit
import plotly.graph_objects as go
import plotly.io as pio
from scipy.ndimage import gaussian_filter
from scipy.ndimage import zoom
import seaborn as sns
import math
from skimage import filters

In [ ]:
def generate_perlin_noise(mapSize, zoom, octaves, persistence, lacunarity, repeatx, repeaty, base):
    shape = (mapSize, mapSize)
    noise_map = np.zeros(shape)

    noise_map = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            x = i / zoom
            y = j / zoom
            noise_map[i][j] = noise.pnoise2(x, y, octaves=octaves, persistence=persistence, lacunarity=lacunarity, repeatx=repeatx, repeaty=repeaty, base=base)
    
    # Normalize to 0-1
    noise_map = (noise_map - noise_map.min()) / (noise_map.max() - noise_map.min())

    return noise_map

def upsample(mat, factor):
    return zoom(mat, factor, order=1)  # order=1 is bilinear

In [ ]:
def ping_pong_erode(heightmap, max_iterations=100, spawn_cycles=50, droplets_per_cycle=1000, depositionRate=0.1, minVol=0.01, evapRate=0.001, waterHeightFactor=0.05, still_water_relaxation=0.3):

    MAX_WATER_PER_CELL = 0.75 # Maximum water volume per cell

    # Neighbor offsets (including diagonals)
    neighbor_offsets = np.array([
        (-1, 1),   # NW
        (0, 1),    # N
        (1, 1),    # NE
        (1, 0),    # E
        (1, -1),   # SE
        (0, -1),   # S
        (-1, -1),  # SW
        (-1, 0),   # W
    ])

    neighbor_distances = np.array([
        np.sqrt(2), # NW
        1.0,        # N
        np.sqrt(2), # NE
        1.0,        # E
        np.sqrt(2), # SE
        1.0,        # S
        np.sqrt(2), # SW
        1.0         # W
    ])

    heightmaps = []
    flowing_watermaps = []
    still_watermaps = []
    sedimentmaps = []
    droplet_spawned_maps = []

    # Some helpers for easier readability
    def _out_of_bounds(nx, ny, dim_x, dim_y):
        """Check if the coordinates (nx, ny) are out of bounds."""
        return nx < 0 or nx >= dim_x or ny < 0 or ny >= dim_y

    def _get_effective_height(x, y, heightmap, watermap):
        """Get the effective height (terrain + water height factor)"""
        return heightmap[y, x] + watermap[y, x] * waterHeightFactor
    
    def _logistic_function(x, L, k, x0):
        """
        A standard logistic function.
        x: input value (slope)
        L: the curve's maximum value
        k: the steepness of the curve
        x0: the x-value of the sigmoid's midpoint
        """
        if x <= 0:
            return 0
        return L / (1 + np.exp(-k * (x - x0)))

    def _spawn_droplets(flowing_water_map, droplet_spawn_map, num_droplets, dim_x, dim_y):
        """Spawn new droplets at random positions"""
        if num_droplets > 0:
            droplets_x = np.random.randint(0, dim_x, size=num_droplets)
            droplets_y = np.random.randint(0, dim_y, size=num_droplets)
            flowing_water_map[droplets_y, droplets_x] += MAX_WATER_PER_CELL * 0.5  # Spawn with half the max water capacity
            droplet_spawn_map[droplets_y, droplets_x] += MAX_WATER_PER_CELL # Used for logging and debugging
            return num_droplets
        return 0

    def _compute_normalized_still_water_weights(x, y, neighbor_offsets, neighbor_distances, current_heightmap, current_still_water_map, waterHeightFactor):
        weights = np.zeros(8, dtype=np.float32)
        total_weight = 0.0

        current_height = current_heightmap[y, x] + current_still_water_map[y, x] * waterHeightFactor

        for i in range(8):
            dx, dy = neighbor_offsets[i]
            nx = x + dx 
            ny = y + dy

            if _out_of_bounds(nx, ny, dim_x, dim_y):
                continue

            neighbor_height = current_heightmap[ny, nx] + current_still_water_map[ny, nx] * waterHeightFactor
            height_diff = current_height - neighbor_height

            if height_diff <= 0.0:
                continue

            distance = neighbor_distances[i]
            slope = height_diff / distance

            weights[i] = slope
            total_weight += slope

        # Normalize weights
        if total_weight > 0.0:
            for i in range(8):
                weights[i] /= total_weight
        else:
            weights.fill(0.0)

        return weights
    
    # Still water still "flows" but not in the same sense as flowing water. It's more like a redistribution of water that has reached a local minimum.
    def _accumulate_still_water_and_sediment(x, y, neighbor_offsets, neighbor_distances, current_heightmap, current_still_water_map, current_still_sediment_map, waterHeightFactor, still_water_relaxation=0.3):
        water_accumulator = 0.0
        sediment_accumulator = 0.0
        current_height = current_heightmap[y, x] + current_still_water_map[y, x] * waterHeightFactor

        for i in range(8):
            dx, dy = neighbor_offsets[i]
            nx = x + dx
            ny = y + dy

            if _out_of_bounds(nx, ny, dim_x, dim_y):
                continue

            neighbor_still_water = current_still_water_map[ny, nx]
            neighbor_height = current_heightmap[ny, nx] + neighbor_still_water * waterHeightFactor
            
            # Skip if the neighbor has no still water or isn't higher than current cell
            if neighbor_still_water <= 0.0 or neighbor_height <= current_height:
                continue

            neighbor_weights = _compute_normalized_still_water_weights(nx, ny, neighbor_offsets, neighbor_distances, current_heightmap, current_still_water_map, waterHeightFactor)
            my_index = (i + 4) % 8
            flow_fraction = neighbor_weights[my_index]

            if flow_fraction > 0.0:
                total_potential_outflow = min(
                    neighbor_still_water,
                    still_water_relaxation * neighbor_still_water * np.sum(neighbor_weights)
                )
                incoming_water = total_potential_outflow * flow_fraction
                water_accumulator += incoming_water

                # Also move a proportional amount of sediment
                if neighbor_still_water > 0:
                    sediment_fraction = incoming_water / neighbor_still_water
                    incoming_sediment = current_still_sediment_map[ny, nx] * sediment_fraction
                    sediment_accumulator += incoming_sediment

        return water_accumulator, sediment_accumulator

    def _compute_normalized_flow_weights(x, y, neighbor_offsets, neighbor_distances, current_heightmap):
        weights = np.zeros(8, dtype=np.float32)
        total_weight = 0.0

        for i in range(8):
            dx, dy = neighbor_offsets[i]
            nx = x + dx 
            ny = y + dy

            if _out_of_bounds(nx, ny, dim_x, dim_y):
                continue

            neighbor_height = current_heightmap[ny, nx]
            height_diff = current_heightmap[y, x] - neighbor_height

            if height_diff <= 0.0:
                continue

            distance = neighbor_distances[i]
            slope = height_diff / distance

            weights[i] = slope
            total_weight += slope

        # Normalize weights
        if total_weight > 0.0:
            for i in range(8):
                weights[i] /= total_weight
        else:
            weights.fill(0.0)

        return weights

    def _accumulate_water_and_sediment(x, y, neighbor_offsets, neighbor_distances, current_heightmap, current_flowing_water_map, current_sediment_map):

        water_accumulator = 0.0
        sediment_accumulator = 0.0
        current_height = current_heightmap[y, x]

        for i in range(8):
            dx, dy = neighbor_offsets[i]
            nx = x + dx 
            ny = y + dy

            if _out_of_bounds(nx, ny, dim_x, dim_y):
                continue

            neighbor_height = current_heightmap[ny, nx]
            neighbor_flowing_water = current_flowing_water_map[ny, nx]
            neighbor_sediment = current_sediment_map[ny, nx]

            # Skip if the neighbor has no water or isn't higher than current cell
            if neighbor_flowing_water <= 0.0 or neighbor_height <= current_height:
                continue

            neighbor_weights = _compute_normalized_flow_weights(nx, ny, neighbor_offsets, neighbor_distances, current_heightmap)
            my_index = (i + 4) % 8  # Opposite direction index
            flow_fraction = neighbor_weights[my_index]

            # If there is flow to current cell (x, y)
            if flow_fraction > 0.0:
                incoming_water = neighbor_flowing_water * flow_fraction
                water_accumulator += incoming_water
                
                # The sediment that flows is simply a fraction of what the neighbor holds
                incoming_sediment = neighbor_sediment * flow_fraction
                sediment_accumulator += incoming_sediment

        # Evaporate incoming water
        water_accumulator *= (1.0 - evapRate)

        return water_accumulator, sediment_accumulator

    dim_x = heightmap.shape[1]
    dim_y = heightmap.shape[0]

    # Generate a random seed for reproducibility
    np.random.seed(42)

    # Generate two copies of the heightmap for ping-pong processing
    heightmap_ping = heightmap.copy()
    heightmap_pong = heightmap.copy()

    # Generate flowing water maps (water that's still moving)
    flowing_water_ping = np.zeros_like(heightmap, dtype=np.float32)
    flowing_water_pong = np.zeros_like(heightmap, dtype=np.float32)
    
    # Generate still water maps (water that has reached local minima)
    still_water_ping = np.zeros_like(heightmap, dtype=np.float32)
    still_water_pong = np.zeros_like(heightmap, dtype=np.float32)
    
    # Generate sediment maps
    sediment_map_ping = np.zeros_like(heightmap, dtype=np.float32)
    sediment_map_pong = np.zeros_like(heightmap, dtype=np.float32)

    # Sediment suspended in still water
    still_sediment_ping = np.zeros_like(heightmap, dtype=np.float32)
    still_sediment_pong = np.zeros_like(heightmap, dtype=np.float32)

    # Calculate total droplets for logging
    total_droplets = spawn_cycles * droplets_per_cycle
    print(f"Will spawn {total_droplets} droplets over {spawn_cycles} cycles ({droplets_per_cycle} per cycle)")
    print(f"Then process for {max_iterations - spawn_cycles} additional iterations")

    for iteration in range(max_iterations):

        current_heightmap = heightmap_ping if iteration % 2 == 0 else heightmap_pong
        current_flowing_water_map = flowing_water_ping if iteration % 2 == 0 else flowing_water_pong
        current_still_water_map = still_water_ping if iteration % 2 == 0 else still_water_pong
        current_sediment_map = sediment_map_ping if iteration % 2 == 0 else sediment_map_pong
        current_still_sediment_map = still_sediment_ping if iteration % 2 == 0 else still_sediment_pong

        next_heightmap = heightmap_pong if iteration % 2 == 0 else heightmap_ping
        next_flowing_water_map = flowing_water_pong if iteration % 2 == 0 else flowing_water_ping
        next_still_water_map = still_water_pong if iteration % 2 == 0 else still_water_ping
        next_sediment_map = sediment_map_pong if iteration % 2 == 0 else sediment_map_ping
        next_still_sediment_map = still_sediment_pong if iteration % 2 == 0 else still_sediment_ping

        # Initialize next state with current state (crucial!)
        next_heightmap[:] = current_heightmap
        next_still_water_map[:] = current_still_water_map  # Still water persists
        next_still_sediment_map[:] = current_still_sediment_map
        next_flowing_water_map[:] = 0.0  # Flowing water gets redistributed each iteration
        next_sediment_map[:] = 0.0  # Sediment gets redistributed each iteration

        droplets_spawned = np.zeros((dim_y, dim_x), dtype=np.float32) # Reset droplets spawned map every iteration

        # Spawn new droplets if we're still in the spawning phase
        if iteration < spawn_cycles:
            spawned = _spawn_droplets(next_flowing_water_map, droplets_spawned, droplets_per_cycle, dim_x, dim_y)
            if iteration == 0:
                print(f"Iteration {iteration}: Spawned {spawned} new droplets")
                print("Initial water sum:", np.sum(next_flowing_water_map))

        for y in range(dim_y):
            for x in range(dim_x):

                # Replace the "find lowest neighbor" section with flow weight calculation
                flow_weights = _compute_normalized_flow_weights(x, y, neighbor_offsets, neighbor_distances, current_heightmap)
                total_flow_weight = np.sum(flow_weights)  # Will be 0.0 if no downslope neighbors

                remobilized_water = 0.0
                remobilized_sediment = 0.0
                deposited_sediment = 0.0
                demobilized_water = 0.0

                # WORKING 🔵
                # Simplified Still Water Re-mobilization Check 💧
                # If a cell with still water now has a downslope path, spill some water over.
                if current_still_water_map[y, x] > 0.0 and total_flow_weight > 0.0:
                    # Remobilize a fraction of the still water, capped by MAX_WATER_PER_CELL
                    # This represents the "spillover" amount for this timestep.
                    actual_remobilize = min(current_still_water_map[y, x] * still_water_relaxation, MAX_WATER_PER_CELL)
                    
                    # Move water from still to flowing
                    next_still_water_map[y, x] -= actual_remobilize
                    remobilized_water = actual_remobilize

                    # Calculate the sediment fraction based on the still water volume
                    sediment_fraction = actual_remobilize / current_still_water_map[y, x]
                    remobilized_sediment = current_still_sediment_map[y, x] * sediment_fraction
                    next_still_sediment_map[y, x] -= remobilized_sediment
                

                # --- STAGE 1: OUTFLOW from the current cell (x,y) ---
                
                # Erosion/Deposition (per cell)
                # Option 1: Cell has downslope neighbors and water (erode terrain + distribute water) 🟢
                # The total water in this cell available to cause erosion and then flow away.
                water_in_cell = current_flowing_water_map[y,x]
                sediment_in_cell = current_sediment_map[y,x]

                # Option 1: Cell has downslope neighbors and water (it will erode and its water will flow away)
                if total_flow_weight > 0.0 and water_in_cell > 0.0:
                    # Calculate average weighted height difference to downslope neighbors
                    avg_height_diff = 0.0
                    for i in range(8):
                        if flow_weights[i] > 0.0:
                            dx, dy = neighbor_offsets[i]
                            nx, ny = x + dx, y + dy
                            neighbor_height = current_heightmap[ny, nx]
                            height_diff = current_heightmap[y, x] - neighbor_height
                            avg_height_diff += height_diff * flow_weights[i]

                    # Erosion capacity is based on the water in this cell
                    erosion_capacity = water_in_cell * avg_height_diff
                    if erosion_capacity < 0.0: 
                        erosion_capacity = 0.0

                    # Calculate sediment change based on the cell's state
                    sdiff = erosion_capacity - sediment_in_cell
                    sediment_change = depositionRate * sdiff

                    # Erode/Deposit at the current grid point. This write is safe.
                    next_heightmap[y, x] -= sediment_change
                    
                    # The sediment that was in this cell is now gone (it flowed away).
                    # The newly eroded/deposited sediment is also gone.
                    # This is implicitly handled because we initialize next_sediment_map to 0.
                    # The _accumulate function will add the correct sediment back in.

                # Option 2: Current cell is the lowest (no downslope neighbors) and has water 🔴
                # (Deposit sediment and evaporate water)
                elif total_flow_weight == 0.0 and water_in_cell > 0.0:
                    # This cell is a pit. Combine the incoming flowing sediment with any
                    # sediment already sitting here in the still map.
                    total_sediment_in_pit = sediment_in_cell + current_still_sediment_map[y, x]

                    # Deposit a fraction of the *total* sediment in the pit.
                    amount_to_deposit = total_sediment_in_pit * depositionRate
                    next_heightmap[y, x] += amount_to_deposit
                    
                    # The remaining sediment stays suspended in the still water.
                    deposited_sediment -= amount_to_deposit
                    
                    # Convert this cell's flowing water to still water and add it to the existing pool.
                    # next_still_water_map[y, x] += water_in_cell * (1.0 - evapRate)
                    demobilized_water = water_in_cell * (1.0 - evapRate)

                    # # Set the next still sediment map to the new, correct amount.
                    # # We use '=' because we've calculated the final state from all sources.
                    # next_still_sediment_map[y, x] = remaining_sediment
                   
                # --- STAGE 2: INFLOW to the current cell (x,y) ---
                
                # This is the "pull" part. Calculate what this cell receives from its neighbors.
                water_accumulator, sediment_accumulator = _accumulate_water_and_sediment(
                    x, y, neighbor_offsets, neighbor_distances, current_heightmap, current_flowing_water_map, current_sediment_map)

                # Total water arriving in this cell for the next step
                total_water = water_accumulator + remobilized_water
                total_sediment = sediment_accumulator + remobilized_sediment

                # Handle water capping and proportional sediment distribution
                excess_water = 0.0
                sediment_to_still = 0.0

                if total_water > MAX_WATER_PER_CELL:
                    excess_water = total_water - MAX_WATER_PER_CELL
                    
                    # Calculate fraction of water that becomes still
                    water_to_still_fraction = excess_water / total_water
                    
                    # Distribute sediment proportionally
                    sediment_to_still = total_sediment * water_to_still_fraction
                    
                    # Adjust flowing sediment
                    sediment_accumulator -= sediment_to_still
                    
                    # Cap the flowing water
                    total_water = MAX_WATER_PER_CELL

                # Always add demobilized water to still water (water that fell into a pit)
                next_still_water_map[y, x] += excess_water + demobilized_water

                # Add sediment to still water: 
                # 1. Sediment that went with excess water
                # 2. Deposited sediment from pits (which is negative, effectively reducing still sediment)
                next_still_sediment_map[y, x] += sediment_to_still + deposited_sediment

                # Update the next water and sediment maps with the pulled-in resources
                next_flowing_water_map[y, x] += total_water
                next_sediment_map[y, x] += sediment_accumulator
                
        
        # This part would be a separate shader pass. Here we are going to ping pong again.
        current_still_water_map = still_water_pong if iteration % 2 == 0 else still_water_ping
        current_still_sediment_map = still_sediment_pong if iteration % 2 == 0 else still_sediment_ping
        next_still_water_map = still_water_ping if iteration % 2 == 0 else still_water_pong
        next_still_sediment_map = still_sediment_ping if iteration % 2 == 0 else still_sediment_pong
        
        next_still_water_map[:] = current_still_water_map
        next_still_sediment_map[:] = current_still_sediment_map

        # Loop over all cells in the
        for y in range(dim_y):
            for x in range(dim_x):
                
                outflow_weights = _compute_normalized_still_water_weights(
                    x, y, neighbor_offsets, neighbor_distances, current_heightmap, current_still_water_map, waterHeightFactor)
                
                total_outflow_weight = np.sum(outflow_weights)

                # Outflow stage
                potential_water_outflow = 0.0
                potential_sediment_outflow = 0.0
                
                if total_outflow_weight > 0.0 and current_still_water_map[y, x] > 0.0:
                    potential_water_outflow = min(
                        current_still_water_map[y, x],
                        still_water_relaxation * current_still_water_map[y, x] * total_outflow_weight
                    )
                    if current_still_water_map[y, x] > 0:
                        sediment_fraction = potential_water_outflow / current_still_water_map[y, x]
                        potential_sediment_outflow = current_still_sediment_map[y, x] * sediment_fraction

                # Inflow stage
                potential_water_inflow, potential_sediment_inflow = _accumulate_still_water_and_sediment(
                    x, y, neighbor_offsets, neighbor_distances, current_heightmap, current_still_water_map, current_still_sediment_map, waterHeightFactor, still_water_relaxation)

                # Net still water and sediment change
                net_water_flow = potential_water_inflow - potential_water_outflow
                net_sediment_flow = potential_sediment_inflow - potential_sediment_outflow
                
                next_still_water_map[y, x] += net_water_flow
                next_still_sediment_map[y, x] += net_sediment_flow

        heightmaps.append(next_heightmap.copy())
        flowing_watermaps.append(next_flowing_water_map.copy())
        still_watermaps.append(next_still_water_map.copy())
        sedimentmaps.append(next_sediment_map.copy())
        # droplet_spawned_maps.append(droplets_spawned.copy())

    return heightmaps, flowing_watermaps, still_watermaps, sedimentmaps, droplet_spawned_maps

In [29]:
L1_iterations, L1_spawn_cycles, L1_droplets_per_cycle = 64, 32, (256*256) // 8 # Note, test the droplets per cycle vs full droplets per cycle
L2_iterations, L2_spawn_cycles, L2_droplets_per_cycle = 32, 16, (128*128) // 8
L3_iterations, L3_spawn_cycles, L3_droplets_per_cycle = 16, 8, (64*64) // 8
L4_iterations, L4_spawn_cycles, L4_droplets_per_cycle = 8, 4, (32*32) // 8

L1_depositionRate = 0.1
L2_depositionRate = 0.2
L3_depositionRate = 0.4
L4_depositionRate = 0.8

# L1_depositionRate = 0.1
# L2_depositionRate = 0.1
# L3_depositionRate = 0.1
# L4_depositionRate = 0.1

minVol = 0.01
evapRate = 0.001

k = 2 # Pointier!

def upsample(mat, factor):
    return zoom(mat, factor, order=3, mode='reflect')  # order=1 is bilinear, order=3 is bicubic

# Uses the Scharr operator for gradient magnitude
def gradient_magnitude(map):
    gx = filters.scharr_h(map)
    gy = filters.scharr_v(map)
    return np.sqrt(gx**2 + gy**2)

# Generate noise at each level with appropriate octaves
noise_L4 = generate_perlin_noise(32, zoom=16, octaves=1, persistence=0.5, lacunarity=2.0, repeatx=32, repeaty=32, base=0)
noise_L3 = generate_perlin_noise(64, zoom=16, octaves=1, persistence=0.5, lacunarity=2.0, repeatx=64, repeaty=64, base=0)
noise_L2 = generate_perlin_noise(128, zoom=16, octaves=1, persistence=0.5, lacunarity=2.0, repeatx=128, repeaty=128, base=0)
noise_L1 = generate_perlin_noise(256, zoom=16, octaves=1, persistence=0.5, lacunarity=2.0, repeatx=256, repeaty=256, base=0)

mag_L4 = gradient_magnitude(noise_L4)
mag_L3 = gradient_magnitude(noise_L3)
mag_L2 = gradient_magnitude(noise_L2)
mag_L1 = gradient_magnitude(noise_L1)

adjusted_L4 = noise_L4 * (1 / (1 + k * mag_L4))
adjusted_L3 = noise_L3 * (1 / (1 + k * (upsample(mag_L4,2) + mag_L3)))
adjusted_L2 = noise_L2 * (1 / (1 + k * (upsample(mag_L4,4) + upsample(mag_L3,2) + mag_L2)))
adjusted_L1 = noise_L1 * (1 / (1 + k * (upsample(mag_L4,8) + upsample(mag_L3,4) + upsample(mag_L2,2) + mag_L1)))

weights = [1.0, 0.5, 0.25, 0.125]

# Erode each level
eroded_L4, flowmap_L4, stillwater_L4, sedimentmap_L4, droplet_spawned_maps_L4 = ping_pong_erode(adjusted_L4,max_iterations=L4_iterations,spawn_cycles=L4_spawn_cycles,droplets_per_cycle=L4_droplets_per_cycle, depositionRate=L4_depositionRate ,minVol=minVol,evapRate=evapRate)
eroded_L4_upscaled = upsample(eroded_L4[-1], 2)
input_L3 = eroded_L4_upscaled + adjusted_L3 * weights[1]
input_L3 = (input_L3 - input_L3.min()) / (input_L3.max() - input_L3.min())

eroded_L3, flowmap_L3, stillwater_L3, sedimentmap_L3, droplet_spawned_maps_L3 = ping_pong_erode(input_L3,max_iterations=L3_iterations,spawn_cycles=L3_spawn_cycles,droplets_per_cycle=L3_droplets_per_cycle, depositionRate=L3_depositionRate ,minVol=minVol,evapRate=evapRate)
eroded_L3_upscaled = upsample(eroded_L3[-1], 2)
input_L2 = eroded_L3_upscaled + adjusted_L2 * weights[2]
input_L2 = (input_L2 - input_L2.min()) / (input_L2.max() - input_L2.min())

eroded_L2, flowmap_L2, stillwater_L2, sedimentmap_L2, droplet_spawned_maps_L2 = ping_pong_erode(input_L2,max_iterations=L2_iterations,spawn_cycles=L2_spawn_cycles,droplets_per_cycle=L2_droplets_per_cycle, depositionRate=L2_depositionRate ,minVol=minVol,evapRate=evapRate)
eroded_L2_upscaled = upsample(eroded_L2[-1], 2)
input_L1 = eroded_L2_upscaled + adjusted_L1 * weights[3]
input_L1 = (input_L1 - input_L1.min()) / (input_L1.max() - input_L1.min())

eroded_L1, flowmap_L1, stillwater_L1, sedimentmap_L1, droplet_spawned_maps_L1 = ping_pong_erode(input_L1,max_iterations=L1_iterations,spawn_cycles=L1_spawn_cycles,droplets_per_cycle=L1_droplets_per_cycle, depositionRate=L1_depositionRate ,minVol=minVol,evapRate=evapRate)


Will spawn 512 droplets over 4 cycles (128 per cycle)
Then process for 4 additional iterations
Iteration 0: Spawned 128 new droplets
Initial water sum: 44.25
Will spawn 4096 droplets over 8 cycles (512 per cycle)
Then process for 8 additional iterations
Iteration 0: Spawned 512 new droplets
Initial water sum: 177.375
Will spawn 32768 droplets over 16 cycles (2048 per cycle)
Then process for 16 additional iterations
Iteration 0: Spawned 2048 new droplets
Initial water sum: 718.5
Will spawn 262144 droplets over 32 cycles (8192 per cycle)
Then process for 32 additional iterations
Iteration 0: Spawned 8192 new droplets
Initial water sum: 2879.625


In [30]:
z = eroded_L1[-1]


print(np.shape(z))
fig = go.Figure(data=[go.Surface(z=z, colorscale='Cividis')])
fig.update_layout(
    title='3D Heightmap after Erosion Simulation',
    autosize=False,
    width=1200,
    height=800,
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Height',
        zaxis=dict(range=[-1, np.max(z)*2]),
        camera=dict(
            eye=dict(x=-0.2, y=1.5, z=0.8)  # Adjust x, y, z for rotation
        )
    )
)
fig.show()

(256, 256)


In [31]:
np.savetxt("Heightmap_gradientTrick_64-32_32-16_16-8_8-4_k=2.csv", z, delimiter=",")